In [1]:
!pip install pyarrow fastparquet huggingface_hub evaluate rouge_score langchain-openai transformers langchain langchainhub langchain-huggingface faiss-cpu langchain-community sentence_transformers

In [2]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

In [5]:
ucsc_passages_df = pd.read_csv("passage.csv", index_col=0)

In [6]:
ucsc_qa_df = pd.read_csv("qa.csv", index_col=0)

In [7]:
model = HuggingFaceHub(
        repo_id="google/flan-t5-base",
        task="text-generation",
        model_kwargs={"temperature": 1, "max_length": 256},
        huggingfacehub_api_token="***REMOVED***"
    )

/home/kyone/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [8]:
ucsc_passge_data_loader = DataFrameLoader(ucsc_passages_df, page_content_column="passage")
ucsc_passage_data = ucsc_passge_data_loader.load()
ucsc_passage_data[0]

Document(metadata={'link': 'https://recreation.ucsc.edu/boating/community-club.html', ' comments': nan}, page_content='Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.')

In [9]:
!pip install -qU langchain-text-splitters

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

docs = text_splitter.split_documents(ucsc_passage_data)
len(docs)

502

In [39]:
embeddings = HuggingFaceEmbeddings()

/home/kyone/miniconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [40]:
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [41]:
index = 23
question = ucsc_qa_df["questions"][index]
print("question:", question)
print("answer:", (ucsc_qa_df["answers"][index]))

question: Is it hard to get help and talk to professors due to large class sizes? 
answer: Almost every course offers in-person faculty office hours so students can get their questions answered and obtain the extra help they may need. Also, larger classes tend to have at least one graduate teaching assistant (TA). TAs often hold office hours and smaller discussion groups to assist students in completing assigned coursework and/or lab work. 


In [42]:
query = question
docs = db.similarity_search(query)
docs

[Document(metadata={'link': 'https://advising.ucsc.edu/planning/your-major/double-major.html', ' comments': nan}, page_content='more classes than are required in their major discipline is just as challenging and rewarding as a double major. Talk with faculty in your major, your academic advisors, and Career Coaches about the benefits and drawbacks of pursuing more than a single major.'),
 Document(metadata={'link': '(Paragraph was generated using ChatGPT) \n\nPrompt: \nI want to answer this question: What are the class sizes for the lower and upper division classes under the engineering school?. Using this website: https://iraps.ucsc.edu/iraps-public-dashboards/instructional-activity/class-sizes.html\n\nWrite a context paragraph for me to answer the question', ' comments': nan}, page_content='To determine the class sizes for lower and upper division classes under the engineering school at UC Santa Cruz, you can refer to the Instructional Activity Dashboard provided by the Institutional

In [49]:
open_ai_llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key="***REMOVED***")
open_ai_llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f858a7684d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f87625c84d0>, root_client=<openai.OpenAI object at 0x7f89291c4490>, root_async_client=<openai.AsyncOpenAI object at 0x7f892919eb50>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

In [58]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()

template = """
Answer the question and provide all relevant information based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    x = "\n\n".join(doc.page_content for doc in docs)
    print(x)
    return x

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | open_ai_llm
    | StrOutputParser()
)

llm_answer = rag_chain.invoke(question)

more classes than are required in their major discipline is just as challenging and rewarding as a double major. Talk with faculty in your major, your academic advisors, and Career Coaches about the benefits and drawbacks of pursuing more than a single major.

To determine the class sizes for lower and upper division classes under the engineering school at UC Santa Cruz, you can refer to the Instructional Activity Dashboard provided by the Institutional Research, Analytics, and Planning Support (IRAPS). This dashboard provides detailed information on class sizes, including cross-listed classes, for different academic divisions and departments. Note that summer quarters are excluded from these calculations. You can access the dashboard for more

that summer quarters are excluded from these calculations. You can access the dashboard for more detailed and specific data on class sizes on this website: https://iraps.ucsc.edu/iraps-public-dashboards/instructional-activity/class-sizes.html

N

In [57]:
print(llm_answer)

The context does not explicitly address the difficulty of getting help and talking to professors due to large class sizes. However, it mentions that the class sizes can vary and suggests that students should consult the Instructional Activity Dashboard for detailed information on class sizes. Since larger class sizes could potentially make it harder to get individual attention from professors, this might imply a challenge, but the context does not provide a definitive answer.


# GPT Answer with recursive text splitter and new prompt
Based on the provided context, it is suggested that class sizes for lower and upper division classes under 
the engineering school at UC Santa Cruz can be found using the Instructional Activity Dashboard. 
While the specifics of class sizes are not detailed in the context, larger class sizes may typically indicate 
more students per professor, which could potentially make it harder to get individualized help and talk to 
professors.

However, the ability to receive help also depends on factors such as faculty availability, office hours, 
and the overall support system in place at the university. It is advisable to engage with faculty, 
academic advisors, and Career Coaches to understand how to navigate the challenges posed by class sizes and to 
explore strategies for getting assistance.

In summary, while larger class sizes can pose challenges to receiving help from professors, the extent of this 
difficulty is not explicitly stated in the context, and resources are available to facilitate communication 
and support.

# GPT Answer with Regular Text splitter and new prompt
'Based on the provided context, it may be challenging to get help and talk to professors due 
to large class sizes, as indicated by the mention of office hours and the preference for faculty,
lecturers, and TAs to hold at least 2-4 hours of office hours per week. Although students are 
encouraged to attend these office hours, the effectiveness of this support can be limited if 
class sizes are large, potentially leading to more students vying for the same limited time with
instructors.\n\nAdditionally, the context notes that a section is a smaller discussion group or 
lab class associated with a credit course, but no credit is awarded, which suggests that not all
lectures require enrollment in these smaller sections. This may further limit opportunities for 
individual interactions with instructors if students are not enrolled in a section.\n\nHowever, 
students do have the option to request appointments outside of regular office hours if they have 
schedule conflicts, which could help mitigate some difficulties in accessing help. Overall, while
there are structured opportunities for assistance, the large number of students may still pose 
challenges in terms of getting individual attention.'

# GPT Answer with recursive text splitter and old prompt
The context does not explicitly address the difficulty of getting help and talking to professors due to large class sizes. 
However, it mentions that the class sizes can vary and suggests that students should consult the Instructional Activity 
Dashboard for detailed information on class sizes. Since larger class sizes could potentially make it harder to get 
individual attention from professors, this might imply a challenge, but the context does not provide a definitive answer.

# Retrieved Documents with Recursive Text Splitter 

more classes than are required in their major discipline is just as challenging and rewarding as a double major
. Talk with faculty in your major, your academic advisors, and Career Coaches about the benefits and drawbacks 
of pursuing more than a single major.

To determine the class sizes for lower and upper division classes under the engineering school at UC Santa Cruz, you can refer to the Instructional Activity Dashboard provided by the Institutional Research, Analytics, and Planning Support (IRAPS). This dashboard provides detailed information on class sizes, including cross-listed classes, for different academic divisions and departments. Note that summer quarters are excluded from these calculations. You can access the dashboard for more

that summer quarters are excluded from these calculations. You can access the dashboard for more detailed and specific data on class sizes on this website: https://iraps.ucsc.edu/iraps-public-dashboards/instructional-activity/class-sizes.html

Note: First-come first-served basis, 10 students maximum capacity based on limited resources. Although the Zoom meeting may appear “open” after these cutoff times, we will only be able to advise you if you entered the waiting room by the posted cutoff time and terms.  If time runs out, you may join the next drop-in session. Please have your UCSC ID card or other photo ID ready.  If you don't have your photo ID, we can only provide general information.

# Retrieved Documents with Character Text Splitter

Faculty, lecturers and TAs should hold office hours to answer students’ questions, preferably at least 2-4 hours a week. To update your office hours, please visit https://accounts.soe.ucsc.edu/directory at your convenience. A list of faculty office hours will automatically post on the SOE web page, http://www.soe.ucsc.edu/people/faculty. If you cannot be present during office hours, it is advised to place a note on your door and/or email instruct@rt.soe.ucsc.edu to post signage for you. Look for updated office hours for the current quarter in our LALS faculty directory or graduate student directory. If your instructor is not a regular LALS faculty member or grad student, you may need to search their name in the campus directory to find their directory information. You may attend office hours any (and every) time they are held. You may also ask for an appointment outside of regular office hours if you have a schedule conflict or if the instructor office hours are listed as by appointment. A section is a smaller discussion group or lab class which is taken as part of a credit course, but no credit is awarded. Secondary sections/labs are listed below the main lecture in Class Search. Not all lectures require enrollment in a secondary section.
https://organization.soe.ucsc.edu/office-hoursabsence-teaching
https://lals.ucsc.edu/undergraduate/office-hours/index.html 
https://registrar.ucsc.edu/soc/enrollment-information.html

To determine the class sizes for lower and upper division classes under the engineering school at UC Santa Cruz, you can refer to the Instructional Activity Dashboard provided by the Institutional Research, Analytics, and Planning Support (IRAPS). This dashboard provides detailed information on class sizes, including cross-listed classes, for different academic divisions and departments. Note that summer quarters are excluded from these calculations. You can access the dashboard for more detailed and specific data on class sizes on this website: https://iraps.ucsc.edu/iraps-public-dashboards/instructional-activity/class-sizes.html

Admit data
Applicants:

71,700
Admits:

46,582
Overall admit rate:

65.0%


GPA of middle 25%-75% students
High School GPA	3.87-4.22

Note: First-come first-served basis, 10 students maximum capacity based on limited resources. Although the Zoom meeting may appear “open” after these cutoff times, we will only be able to advise you if you entered the waiting room by the posted cutoff time and terms.  If time runs out, you may join the next drop-in session. Please have your UCSC ID card or other photo ID ready.  If you don't have your photo ID, we can only provide general information.

## Old Prompt
"""
Answer the question based only on the following context:
{context}

Question: {question}
"""

## New Prompt
"""
Answer the question and provide all relevant information based only on the following context:
{context}

Question: {question}
"""